In [ ]:
import pandas as pd
import re

# Load data
df = pd.read_csv("Metadata/metadata.csv", dtype=str)
info = pd.read_csv("Metadata/comInfo.csv", dtype=str)
info = info[info["종목코드"].notna()].reset_index(drop=True)

df.shape, info.shape

In [ ]:
def get_time_end_cycle(report_name):
    try:
        # Create pattern for YYYY.MM
        temp = re.findall(r"\d{4}\.\d{2}", report_name)
        assert len(temp) == 1
        return temp[0]
    except: return None

# Get time_end_cycle, then filter out the rows with None
df["time_end_cycle"] = df["보고서명"].apply(get_time_end_cycle)
df = df[df["time_end_cycle"].notna()].reset_index(drop=True)
df["time_filled"] = df["접수일자"].str[:7]
df.shape

In [ ]:
# Map popup_id to company_code, then filter out the rows with None
dict_popup_id_to_code = dict(zip(info["popup_id"], info["종목코드"]))
df["company_code"] = df["popup_href"].map(dict_popup_id_to_code)
df = df[df["company_code"].notna()].reset_index(drop=True)
df.shape

In [ ]:
def get_time_info(date_filled):
    year = int(date_filled[:4])
    month = int(date_filled[5:7])
    day = int(date_filled[8:])

    if month < 4 or (month == 4 and day <= 10):
        return year - 1
    return year

# Get time_info, if the month is before the 10th April, then the year is the previous year, else the year is the current year
# time_invest is April of the year after time_info
df["time_info"] = df["접수일자"].apply(get_time_info)
df["time_invest"] = df["time_info"].apply(lambda x: str(x+1) + ".04")
df.head()

In [ ]:
def get_month_diff(time_invest, time_end_cycle):
    a, b = time_invest.split(".")
    c, d = time_end_cycle.split(".")
    return int(a) * 12 + int(b) - int(c) * 12 - int(d)

# Get the month difference between time_invest and time_end_cycle, then filter out the rows with month_diff <= 0 and > 15
df["month_diff"] = df.apply(lambda x: get_month_diff(x["time_invest"], x["time_end_cycle"]), axis=1)
df = df[(df["month_diff"] > 0) & (df["month_diff"] <= 15)].reset_index(drop=True)
df.shape

In [ ]:
# Filter out the rows with the date range
start_date = "2010.04.11"
end_date = "2024.04.10"

df = df[(df["접수일자"] >= start_date) & (df["접수일자"] <= end_date)].reset_index(drop=True)
df.shape

In [ ]:
# Drop duplicates by company_code and time_info, keep the first
df.drop_duplicates(["company_code", "time_info"], keep="first", inplace=True, ignore_index=True)
df.shape

In [8]:
df.to_csv("Metadata/filtered_2010_2023.csv", index=False)